# PHE Download

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Printable Class

Simple class that allows other classes to be printed.

In [2]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Configuration

Data to download via the API - cases, patients, deaths

In [3]:
ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"

In [4]:
casesStructure = {
    "cases": "newCasesBySpecimenDate", # Cases by specimen date
    "casesRollingSum": "newCasesBySpecimenDateRollingSum", # Total cases (7-day average)
    "casesRollingRate": "newCasesBySpecimenDateRollingRate", # Rate of cases per 100K over 7 days
    "casesAgeDemographics": "newCasesBySpecimenDateAgeDemographics", # Demographics
    "casesReported": "newCasesByPublishDate" # Cases by date reported
}

patientsStructure = {
    "hospitalAdmissions": "newAdmissions", # Patients admitted to hospital
    "hospitalPatients": "hospitalCases", # Patients in hospital
    "hospitalPatientsMv": "covidOccupiedMVBeds" # Patients in mechanical ventilation beds
}

deathsStructure = {
    "deaths": "newDeaths28DaysByDeathDate", # Deaths within 28d of +ve test by date of death
    "deathsRollingSum": "newDeaths28DaysByDeathDateRollingSum", # Total deaths by date of death (7-day average)
    "deathsRollingRate": "newDeaths28DaysByDeathDateRollingRate", # Rate of deaths per 100K over 7 days
    "deathsAgeDemographics": "newDeaths28DaysByDeathDateAgeDemographics", # Demographics
    "deathsReported": "newDeaths28DaysByPublishDate" # Deaths within 28d of +ve test by date reported
}

onsStructure = {
    "deathsRegistered": "newOnsDeathsByRegistrationDate" # COVID-19 on the death certificate
}

In [5]:
ageDemographics = [
    '00_04',
    '05_09',
    '10_14',
    '15_19',
    '20_24',
    '25_29',
    '30_34',
    '35_39',
    '40_44',
    '45_49',
    '50_54',
    '55_59',
    '60_64',
    '65_69',
    '70_74',
    '75_79',
    '80_84',
    '85_89',
    '90+'    
]

## Area Class

Download data via the API and prepare it for analysis.

Supports nations, regions and LTLAs.

In [6]:
import requests
import json
import csv

class Area(Printable):
    def __init__(self, areaType, areaName):
        """Initialisise the area object"""

        self.areaType = areaType
        self.areaName = areaName
        self.safeName = areaName.lower().replace(' ', '_').replace(',', '')
        for word in 'of', 'and', 'the':
            self.safeName = self.safeName.replace(word + '_', '')
        self.csvName = self.safeName + '.csv'
        

    def getRawPath(self, period):
        """Get path for raw data"""
        rawPath = os.path.join(projdir, "data", "phe-dashboard", "raw", period, self.areaType)

        return rawPath

        
    def getCsvPath(self, period, category):
        """Get path for csv data"""
        csvPath = os.path.join(projdir, "data", "phe-dashboard", "csv", period, category, self.areaType)

        return csvPath

        
    def download(self, period = "daily"):
        """Download data from PHE dashboard"""

        # Catch all exceptions
        try:
            filters = [
                f"areaType={self.areaType}",
                f"areaName={self.areaName}"
            ]

            structure = {
                "date": "date",
                "areaName": "areaName"
            }

            if period == "weekly":
                if self.areaType in ['nation', 'region', 'ltla']:
                    structure.update(onsStructure)
            else:
                if self.areaType in ['nation', 'region', 'ltla']:
                    structure.update(casesStructure)
                    structure.update(deathsStructure)
                if self.areaType in ['nation', 'nhsregion']:
                    structure.update(patientsStructure)

            api_params = {
                "filters": str.join(";", filters),
                "structure": json.dumps(structure, separators=(",", ":")),
                "format": "csv"
            }

            # Download raw data - hybrid of CSV and Python dictionaries
            response = requests.get(ENDPOINT, params=api_params, timeout=10)
            assert response.status_code == 200, f"Failed request for {self.areaName}: {response.status_code} {response.text}"

            # Ensure raw path exists
            rawPath = self.getRawPath(period)
            if not os.path.exists(rawPath):
                os.makedirs(rawPath)

            # Save raw data
            rawFn = os.path.join(rawPath, self.csvName)
            with open(rawFn, 'w') as f:
                f.write(response.content.decode())

        # General catch all to report exceptions then abort
        except:
            print(f"Failed to download {period} data for {self.areaName}")
            raise


    def downloadDaily(self):
        """Download daily data for analysis"""

        print(f"Downloading {self.areaName}...")
        self.download()


    def downloadWeekly(self):
        """Download weekly data for analysis"""

        if self.areaType in ['nation', 'region', 'ltla']:
            print(f"Downloading {self.areaName}...")
            self.download("weekly")


    def prepare(self, category, period = "daily"):
        """Prepare data for analysis"""

        def getColNos(row, category):
            """Get column numbers relevant to the category"""

            if category == "cases":
                structure = casesStructure
            elif category == "patients":
                structure = patientsStructure
            elif category == "deaths":
                structure = deathsStructure
            elif category == "ons":
                structure = onsStructure
            else:
                raise ValueError(f"Unsupported category - {category}") 

            # Always include date and areaName
            colNos = [0, 1]

            # Other columns are dependent on the category and its structure
            for colNo in range(len(row)):
                if row[colNo] in structure:
                    if row[colNo].endswith("Demographics"):
                        colNos.append(colNo)
                    else:
                        colNos.append(colNo)
                
            return colNos


        def getColNames(row, colNos):
            """Get column names"""

            # Always include date and areaName
            colNames = []

            # Other columns are dependent on the category and its structure
            for colNo in colNos:
                if row[colNo].endswith("Demographics"):
                    for ageDemographic in ageDemographics:
                        colNames.append(f"{category}{ageDemographic}")
                        colNames.append(f"{category}RollingSum{ageDemographic}")
                        colNames.append(f"{category}RollingRate{ageDemographic}")
                else:
                    colNames.append(row[colNo])
                
            return colNames


        def getRowValues(row, colNos):
            """Get row values"""

            tidyRow = []
            for colNo in colNos:
                if row[colNo].startswith('['):
                    items = eval(row[colNo])
                    for ageDemographic in ageDemographics:
                        found = False
                        for item in items:
                            if item["age"] == ageDemographic:
                                tidyRow.append(item[category])
                                tidyRow.append(item["rollingSum"])
                                tidyRow.append(item["rollingRate"])
                                found = True
                                break

                        if found == False:
                            tidyRow.append("")
                            tidyRow.append("")
                            tidyRow.append("")
                else:
                    tidyRow.append(row[colNo])

            return tidyRow


        # Catch all exceptions
        try:
            # Determine raw filename
            rawPath = self.getRawPath(period)
            rawFn = os.path.join(rawPath, self.csvName)

            # Ensure the CSV path exists
            csvPath = self.getCsvPath(period, category)
            if not os.path.exists(csvPath):
                os.makedirs(csvPath)

            # Generate the CSV from raw data
            csvFn = os.path.join(csvPath, self.csvName)
            with open(csvFn, 'w') as csvFile:
                writer = csv.writer(csvFile)
                with open(rawFn, 'r') as f:
                    reader = csv.reader(f, delimiter = ',')
                    rows = []
                    rowNo = 0
                    for row in reader:
                        if rowNo == 0:
                            colNos = getColNos(row, category)
                            colNames = getColNames(row, colNos)
                        else:
                            row = getRowValues(row, colNos)
                            rows.append(row)

                        rowNo += 1

                    rows.reverse()
                    writer.writerow(colNames)
                    writer.writerows(rows)

        # General catch all to report exceptions then abort
        except:
            print(f"Failed to convert {period} {category} for {self.areaName}")
            raise


    def prepareDaily(self):
        """Prepare daily data for analysis"""

        print(f"Preparing {self.areaName}...")

        if self.areaType in ["nation", "region", "ltla"]:
            self.prepare("cases")
            self.prepare("deaths")
        if self.areaType in ["nation", "nhsregion"]:
            self.prepare("patients")


    def prepareWeekly(self):
        """Prepare weekly data for analysis"""

        if self.areaType in ["nation", "region", "ltla"]:
            print(f"Preparing {self.areaName}...")
            self.prepare("ons", "weekly")

In [7]:
nationNames = ["England"]

regionNames = ["North West", "North East", "Yorkshire and The Humber", "West Midlands", "East Midlands",
             "East of England", "London", "South East", "South West"]

nhsRegionNames = ["North West", "North East and Yorkshire", "Midlands",
                  "East of England", "London", "South East", "South West"]

ltlaNames = ["Dorset", "Bournemouth, Christchurch and Poole",
             "Stevenage", "Welwyn Hatfield", "North Hertfordshire", "East Hertfordshire",
             "Sandwell", "Dudley", "Birmingham",
             "Derbyshire Dales", "North East Derbyshire", "High Peak", "Sheffield",
             "Croydon"]

areas = [("nation", ["England"]), ("region", regionNames), ("nhsregion", nhsRegionNames), ("ltla", ltlaNames)]

In [8]:
print("--- Daily Data ---")

for areaType, areaNames in areas:
    for areaName in areaNames:
        area = Area(areaType, areaName)
        area.downloadDaily()
        
print("\n--- Weekly Data ---")

for areaType, areaNames in areas:
    for areaName in areaNames:
        area = Area(areaType, areaName)
        area.downloadWeekly()
        
print("\nAll done!")

--- Daily Data ---

--- Weekly Data ---

All done!


In [9]:
print("--- Preparing daily data ---")

for areaType, areaNames in areas:
    for areaName in areaNames:
        area = Area(areaType, areaName)
        area.prepareDaily()

print("\n---Preparing weekly data ---")

for areaType, areaNames in areas:
    for areaName in areaNames:
        area = Area(areaType, areaName)
        area.prepareWeekly()
        
print("\nAll done!")

--- Preparing daily data ---
Preparing England...
Preparing North West...
Preparing North East...
Preparing Yorkshire and The Humber...
Preparing West Midlands...
Preparing East Midlands...
Preparing East of England...
Preparing London...
Preparing South East...
Preparing South West...
Preparing North West...
Preparing North East and Yorkshire...
Preparing Midlands...
Preparing East of England...
Preparing London...
Preparing South East...
Preparing South West...
Preparing Dorset...
Preparing Bournemouth, Christchurch and Poole...
Preparing Stevenage...
Preparing Welwyn Hatfield...
Preparing North Hertfordshire...
Preparing East Hertfordshire...
Preparing Sandwell...
Preparing Dudley...
Preparing Birmingham...
Preparing Derbyshire Dales...
Preparing North East Derbyshire...
Preparing High Peak...
Preparing Sheffield...
Preparing Croydon...

---Preparing weekly data ---
Preparing England...
Preparing North West...
Preparing North East...
Preparing Yorkshire and The Humber...
Preparing W

In [10]:
print("All done!")

All done!
